#인기있는 tv 프로그램 중 장르 id가 10762(kids) 인 리스트 정보만 불러오기

In [6]:
from langdetect import detect
import requests

# TMDB API 키
TMDB_API_KEY = 'cd88d12bfe4c5d842ef9a464b2f0bcd1'
language = 'ko'

#TV SERIES LISTS > POPULAR
#SERIES_ID
def fetch_popular_series(genre_id=10762, language='ko-KR', region='KR', page=1, total_results=200):
    url = 'https://api.themoviedb.org/3/discover/tv'
    params = {
        'api_key': TMDB_API_KEY,
        'language': language,
        'region': region,
        'page': page,
        'with_genres': genre_id
    }

    results = []
    while len(results) < total_results:
        response = requests.get(url, params=params)
        data = response.json()
        for series in data['results']:
            results.append(series)
            if len(results) >= total_results:
                break
        page += 1
        params['page'] = page

    return results[:total_results]

#TV SERIES > CONTENT_RATING
#RATING
def fetch_series_rating(series_id):
    url = f'https://api.themoviedb.org/3/tv/{series_id}/content_ratings'
    params = {
        'api_key': TMDB_API_KEY
    }

    response = requests.get(url, params=params)
    data = response.json()
    
    kr_rating = None
    de_rating = None
    
    if 'results' in data:
        for result in data['results']:
            iso_3166_1 = result.get('iso_3166_1')
            rating = result.get('rating')
            
            if iso_3166_1 == 'KR':
                kr_rating = rating
            elif iso_3166_1 == 'DE':
                de_rating = rating
    
    if kr_rating:
        return kr_rating
    elif de_rating:
        return de_rating
    else:
        return None
    
#TV SERIES > DETAILS
#CAST, CREW
def fetch_series_details(series_id, language='ko-KR'):
    url = f'https://api.themoviedb.org/3/tv/{series_id}'
    params = {
        'api_key': TMDB_API_KEY,
        'language': language,
        'append_to_response': 'credits'  # 출연진 및 제작진 정보
    }

    response = requests.get(url, params=params)
    data = response.json()

    if 'overview' not in data or not data['overview']:
        params['language'] = 'en'
        response = requests.get(url, params=params)
        data = response.json()

    return data

#TV SERIES > IMAGES
#LOGO, POSTER
def fetch_series_images(series_id, language='ko'):
    url = f'https://api.themoviedb.org/3/tv/{series_id}/images'
   
    params = {
        'api_key': TMDB_API_KEY,
        'language': language
    }

    response = requests.get(url, params=params)
    data = response.json()
    
    # 한국어 버전 없을 시 영어버전 출력
    if ('posters' not in data or len(data['posters']) == 0) or ('logos' not in data or len(data['logos']) == 0):
        params['language'] = 'en'
        response = requests.get(url, params=params)
        data = response.json()

        if 'posters' in data and len(data['posters']) > 0:
            data['posters'] = data['posters'][:1]
        if 'logos' in data and len(data['logos']) > 0:
            data['logos'] = data['logos'][:1]

    return data

#TV SERIES > VIDEOS
#TRAILER 
def fetch_series_videos(series_id, language='en'):
    url = f'https://api.themoviedb.org/3/tv/{series_id}/videos'

    params = {
        'api_key': TMDB_API_KEY,
        'language': language,
    }
    response = requests.get(url, params=params)
    data = response.json()

    key_mapping = {
        'Trailer': 'trailer',
        'Clip': 'clip',
        'Teaser': 'teaser',
        'Opening Credits': 'opening_credits'
    }

    for item in data.get('results', []):
        video_type = item.get('type')
        if video_type in key_mapping:
            return item.get('key')

    return None


def print_series_info(series, series_images, series_videos):

    #01. SERIES_ID
    print("id:", series['id'])

    ##. First_air_date
    print("first_air_date", series['first_air_date'])

    #02. SUB_CATEGORY 
    #고정값

    #03. TITLE
    print("제목:", series['name'])

    #04. GENRE
    if 'genres' in series:
        if len(series['genres']) == 1: # 장르 내 리스트가 하나뿐인 경우에는 언어 상관없이 출력
            genre = series['genres'][0]['name']
        else:
            genres = [genre['name'] for genre in series['genres'] if detect(genre['name']) == 'ko']
            genre = ", ".join(genres)
    else:
        genre = None
    
    
    #05. RATING
    series_id = series['id']
    rating = fetch_series_rating(series_id)
    print("등급:", rating)

    #06. SEASON_NUM
    print("시즌 수:", f"{series['number_of_seasons']}")

    #07. OVERVIEW
    print("줄거리:", series['overview'])
    

    #08. CAST
    if 'credits' in series and 'cast' in series['credits']:
        print("출연진:")
        for cast in series['credits']['cast'][:3]:
            print(f"{cast['name']}")
    
    #09. CREW
    if 'credits' in series and 'crew' in series['credits']:
        print("제작진:")
        if series['credits']['crew']:  # 제작진 정보가 있는 경우에만 출력
            crew = series['credits']['crew'][0]  # 첫 번째 제작진 정보만 출력
            print(f"{crew['name']}")
    
    #10. LOGO
    if 'logos' in series_images:
        for logo in series_images['logos']:
            print("로고 URL:", f"https://image.tmdb.org/t/p/original{logo['file_path']}")

    #11. POSTER_URL
    if 'posters' in series_images:
         for poster in series_images['posters']:
            print("포스터 URL:", f"https://image.tmdb.org/t/p/original{poster['file_path']}")

    #12. TRAILER_URL
    if series_videos is not None:  
        print("예고편 URL:", f"https://www.youtube.com/watch?v={series_videos}")


def main():
    popular_tv_series = fetch_popular_series()

    for series in popular_tv_series:
        series_details = fetch_series_details(series['id'])
        series_images = fetch_series_images(series['id'])
        series_videos = fetch_series_videos(series['id']) 
        print_series_info(series_details, series_images, series_videos) 
        print()

if __name__ == "__main__":
    main()


id: 1848
first_air_date 2004-01-28
제목: Winx Club
등급: 6
시즌 수: 8
줄거리: In a magical universe, witches, warriors begin fighting in the name of good .vs. evil! At a magic school, five teenage girls are selected to defend the universe with their magic.
출연진:
Letizia Ciampa
Perla Liberatori
Gemma Donati
제작진:
로고 URL: https://image.tmdb.org/t/p/original/ivbWJcKJCCLqYdV61aQX5Z6kj0s.png
예고편 URL: https://www.youtube.com/watch?v=S35nFJH8e4Y

id: 4574
first_air_date 1992-10-31
제목: 엑스맨
등급: 16
시즌 수: 5
줄거리: 마블 코믹스 세계에서, 유전적으로 부여된 초능력을 가진 사람인 돌연변이는 증오와 두려움에 가득 찬 대중에게 박해를 받는다. 이에 대한  피난처 중 하나는 자비에르 교수의 영재 아카데미이다.
출연진:
Cedric Smith
Norm Spencer
Catherine Disher
제작진:
Leanne Moreau
로고 URL: https://image.tmdb.org/t/p/original/sj8UwAX0wjx1Xu6G1NTWLlMG9py.png
포스터 URL: https://image.tmdb.org/t/p/original/jRGOY8KvL7hHrJcfKfXe99C4pc5.jpg
예고편 URL: https://www.youtube.com/watch?v=gh0KJImjkqc

id: 2808
first_air_date 2001-11-03
제목: Totally Spies!
등급: 16
시즌 수: 7
줄거리: Totally Spies! depicts three girlfriends 'with an a

KeyboardInterrupt: 

#first_air_date가 2000년 이전인 것 제외 => 실패함

In [5]:
from langdetect import detect
import requests

# TMDB API 키
TMDB_API_KEY = 'cd88d12bfe4c5d842ef9a464b2f0bcd1'
language = 'ko'

#TV SERIES LISTS > POPULAR
#SERIES_ID
from datetime import datetime

def fetch_specific_genre_series(genre_id=10762, language='ko-KR', region='KR', page=1, total_results=200):
    url = 'https://api.themoviedb.org/3/discover/tv'
    params = {
        'api_key': TMDB_API_KEY,
        'language': language,
        'region': region,
        'page': page,
        'with_genres': genre_id,
    }

    results = []
    while len(results) < total_results:
        response = requests.get(url, params=params)
        data = response.json()
        for series in data['results']:
            # "first_air_date" 값이 2000년 이후인 경우에만 결과에 추가
            first_air_date = series.get('first_air_date')
            if first_air_date and datetime.strptime(first_air_date, '%Y-%m-%d') >= datetime(2000, 1, 1):
                results.append(series)
                if len(results) >= total_results:
                    break
        page += 1
        params['page'] = page

    return results[:total_results]


#TV SERIES > CONTENT_RATING
#RATING
def fetch_series_rating(series_id):
    url = f'https://api.themoviedb.org/3/tv/{series_id}/content_ratings'
    params = {
        'api_key': TMDB_API_KEY
    }

    response = requests.get(url, params=params)
    data = response.json()
    
    kr_rating = None
    de_rating = None
    
    if 'results' in data:
        for result in data['results']:
            iso_3166_1 = result.get('iso_3166_1')
            rating = result.get('rating')
            
            if iso_3166_1 == 'KR':
                kr_rating = rating
            elif iso_3166_1 == 'DE':
                de_rating = rating
    
    if kr_rating:
        return kr_rating
    elif de_rating:
        return de_rating
    else:
        return None
    
#TV SERIES > DETAILS
#CAST, CREW
def fetch_series_details(series_id, language='ko-KR'):
    url = f'https://api.themoviedb.org/3/tv/{series_id}'
    params = {
        'api_key': TMDB_API_KEY,
        'language': language,
        'append_to_response': 'credits'  # 출연진 및 제작진 정보
    }

    response = requests.get(url, params=params)
    data = response.json()

    if 'overview' not in data or not data['overview']:
        params['language'] = 'en'
        response = requests.get(url, params=params)
        data = response.json()

    return data

#TV SERIES > IMAGES
#LOGO, POSTER
def fetch_series_images(series_id, language='ko'):
    url = f'https://api.themoviedb.org/3/tv/{series_id}/images'
   
    params = {
        'api_key': TMDB_API_KEY,
        'language': language
    }

    response = requests.get(url, params=params)
    data = response.json()
    
    # 한국어 버전 없을 시 영어버전 출력
    if ('posters' not in data or len(data['posters']) == 0) or ('logos' not in data or len(data['logos']) == 0):
        params['language'] = 'en'
        response = requests.get(url, params=params)
        data = response.json()

        if 'posters' in data and len(data['posters']) > 0:
            data['posters'] = data['posters'][:1]
        if 'logos' in data and len(data['logos']) > 0:
            data['logos'] = data['logos'][:1]

    return data

#TV SERIES > VIDEOS
#TRAILER 
def fetch_series_videos(series_id, language='en'):
    url = f'https://api.themoviedb.org/3/tv/{series_id}/videos'

    params = {
        'api_key': TMDB_API_KEY,
        'language': language,
    }
    response = requests.get(url, params=params)
    data = response.json()

    key_mapping = {
        'Trailer': 'trailer',
        'Clip': 'clip',
        'Teaser': 'teaser',
        'Opening Credits': 'opening_credits'
    }

    for item in data.get('results', []):
        video_type = item.get('type')
        if video_type in key_mapping:
            return item.get('key')

    return None


def print_series_info(series, series_images, series_videos):

    #01. SERIES_ID
    print("id:", series['id'])

    ##. First_air_date
    print("first_air_date", series['first_air_date'])

    #02. SUB_CATEGORY 
    #고정값

    #03. TITLE
    print("제목:", series['name'])

    #04. GENRE
    if 'genres' in series:
        if len(series['genres']) == 1: # 장르 내 리스트가 하나뿐인 경우에는 언어 상관없이 출력
            genre = series['genres'][0]['name']
        else:
            genres = [genre['name'] for genre in series['genres'] if detect(genre['name']) == 'ko']
            genre = ", ".join(genres)
    else:
        genre = None
    
    #05. RATING
    series_id = series['id']
    rating = fetch_series_rating(series_id)
    print("등급:", rating)

    #06. SEASON_NUM
    print("시즌 수:", f"{series['number_of_seasons']}")

    #07. OVERVIEW
    print("줄거리:", series['overview'])
    

    #08. CAST
    if 'credits' in series and 'cast' in series['credits']:
        print("출연진:")
        for cast in series['credits']['cast'][:3]:
            print(f"{cast['name']}")
    
    #09. CREW
    if 'credits' in series and 'crew' in series['credits']:
        print("제작진:")
        if series['credits']['crew']:  # 제작진 정보가 있는 경우에만 출력
            crew = series['credits']['crew'][0]  # 첫 번째 제작진 정보만 출력
            print(f"{crew['name']}")
    
    #10. LOGO
    if 'logos' in series_images:
        for logo in series_images['logos']:
            print("로고 URL:", f"https://image.tmdb.org/t/p/original{logo['file_path']}")

    #11. POSTER_URL
    if 'posters' in series_images:
         for poster in series_images['posters']:
            print("포스터 URL:", f"https://image.tmdb.org/t/p/original{poster['file_path']}")

    #12. TRAILER_URL
    if series_videos is not None:  
        print("예고편 URL:", f"https://www.youtube.com/watch?v={series_videos}")


def main():
    popular_tv_series = fetch_popular_series()

    for series in popular_tv_series:
        series_details = fetch_series_details(series['id'])
        series_images = fetch_series_images(series['id'])
        series_videos = fetch_series_videos(series['id']) 
        print_series_info(series_details, series_images, series_videos) 
        print()

if __name__ == "__main__":
    main()


id: 1848
first_air_date 2004-01-28
제목: Winx Club
등급: 6
시즌 수: 8
줄거리: In a magical universe, witches, warriors begin fighting in the name of good .vs. evil! At a magic school, five teenage girls are selected to defend the universe with their magic.
출연진:
Letizia Ciampa
Perla Liberatori
Gemma Donati
제작진:
로고 URL: https://image.tmdb.org/t/p/original/ivbWJcKJCCLqYdV61aQX5Z6kj0s.png
예고편 URL: https://www.youtube.com/watch?v=S35nFJH8e4Y

id: 4574
first_air_date 1992-10-31
제목: 엑스맨
등급: 16
시즌 수: 5
줄거리: 마블 코믹스 세계에서, 유전적으로 부여된 초능력을 가진 사람인 돌연변이는 증오와 두려움에 가득 찬 대중에게 박해를 받는다. 이에 대한  피난처 중 하나는 자비에르 교수의 영재 아카데미이다.
출연진:
Cedric Smith
Norm Spencer
Catherine Disher
제작진:
Leanne Moreau
로고 URL: https://image.tmdb.org/t/p/original/sj8UwAX0wjx1Xu6G1NTWLlMG9py.png
포스터 URL: https://image.tmdb.org/t/p/original/jRGOY8KvL7hHrJcfKfXe99C4pc5.jpg
예고편 URL: https://www.youtube.com/watch?v=gh0KJImjkqc

id: 2808
first_air_date 2001-11-03
제목: Totally Spies!
등급: 16
시즌 수: 7
줄거리: Totally Spies! depicts three girlfriends 'with an a

KeyboardInterrupt: 

#인기 tv프로그램 200개 > 장르가 kids > 컨텐츠 제작 국가가 한국인 것 출력

In [8]:
from langdetect import detect
import requests

# TMDB API 키
TMDB_API_KEY = 'cd88d12bfe4c5d842ef9a464b2f0bcd1'
language = 'ko'

#TV SERIES LISTS > POPULAR
#SERIES_ID
def fetch_popular_series(genre_id=10762, language='ko-KR', region='KR', page=1, total_results=200):
    url = 'https://api.themoviedb.org/3/discover/tv'
    params = {
        'api_key': TMDB_API_KEY,
        'language': language,
        'region': region,
        'page': page,
        'with_genres': genre_id
    }

    results = []
    while len(results) < total_results:
        response = requests.get(url, params=params)
        data = response.json()
        for series in data['results']:
            # origin_country가 KR을 포함하는 경우에만 결과에 추가
            if 'KR' in series.get('origin_country', []):
                results.append(series)
                if len(results) >= total_results:
                    break
        page += 1
        params['page'] = page

    return results[:total_results]


#TV SERIES > CONTENT_RATING
#RATING
def fetch_series_rating(series_id):
    url = f'https://api.themoviedb.org/3/tv/{series_id}/content_ratings'
    params = {
        'api_key': TMDB_API_KEY
    }

    response = requests.get(url, params=params)
    data = response.json()
    
    kr_rating = None
    de_rating = None
    
    if 'results' in data:
        for result in data['results']:
            iso_3166_1 = result.get('iso_3166_1')
            rating = result.get('rating')
            
            if iso_3166_1 == 'KR':
                kr_rating = rating
            elif iso_3166_1 == 'DE':
                de_rating = rating
    
    if kr_rating:
        return kr_rating
    elif de_rating:
        return de_rating
    else:
        return None
    
#TV SERIES > DETAILS
#CAST, CREW
def fetch_series_details(series_id, language='ko-KR'):
    url = f'https://api.themoviedb.org/3/tv/{series_id}'
    params = {
        'api_key': TMDB_API_KEY,
        'language': language,
        'append_to_response': 'credits'  # 출연진 및 제작진 정보
    }

    response = requests.get(url, params=params)
    data = response.json()

    if 'overview' not in data or not data['overview']:
        params['language'] = 'en'
        response = requests.get(url, params=params)
        data = response.json()

    return data

#TV SERIES > IMAGES
#LOGO, POSTER
def fetch_series_images(series_id, language='ko'):
    url = f'https://api.themoviedb.org/3/tv/{series_id}/images'
   
    params = {
        'api_key': TMDB_API_KEY,
        'language': language
    }

    response = requests.get(url, params=params)
    data = response.json()
    
    # 한국어 버전 없을 시 영어버전 출력
    if ('posters' not in data or len(data['posters']) == 0) or ('logos' not in data or len(data['logos']) == 0):
        params['language'] = 'en'
        response = requests.get(url, params=params)
        data = response.json()

        if 'posters' in data and len(data['posters']) > 0:
            data['posters'] = data['posters'][:1]
        if 'logos' in data and len(data['logos']) > 0:
            data['logos'] = data['logos'][:1]

    return data

#TV SERIES > VIDEOS
#TRAILER 
def fetch_series_videos(series_id, language='en'):
    url = f'https://api.themoviedb.org/3/tv/{series_id}/videos'

    params = {
        'api_key': TMDB_API_KEY,
        'language': language,
    }
    response = requests.get(url, params=params)
    data = response.json()

    key_mapping = {
        'Trailer': 'trailer',
        'Clip': 'clip',
        'Teaser': 'teaser',
        'Opening Credits': 'opening_credits'
    }

    for item in data.get('results', []):
        video_type = item.get('type')
        if video_type in key_mapping:
            return item.get('key')

    return None


def print_series_info(series, series_images, series_videos):

    #01. SERIES_ID
    print("id:", series['id'])

    ##. First_air_date
    print("first_air_date", series['first_air_date'])

    #02. SUB_CATEGORY 
    #고정값

    #03. TITLE
    print("제목:", series['name'])

    #04. GENRE
    if 'genres' in series:
        if len(series['genres']) == 1: # 장르 내 리스트가 하나뿐인 경우에는 언어 상관없이 출력
            genre = series['genres'][0]['name']
        else:
            genres = [genre['name'] for genre in series['genres'] if detect(genre['name']) == 'ko']
            genre = ", ".join(genres)
    else:
        genre = None
    
    
    #05. RATING
    series_id = series['id']
    rating = fetch_series_rating(series_id)
    print("등급:", rating)

    #06. SEASON_NUM
    print("시즌 수:", f"{series['number_of_seasons']}")

    #07. OVERVIEW
    print("줄거리:", series['overview'])
    

    #08. CAST
    if 'credits' in series and 'cast' in series['credits']:
        print("출연진:")
        for cast in series['credits']['cast'][:3]:
            print(f"{cast['name']}")
    
    #09. CREW
    if 'credits' in series and 'crew' in series['credits']:
        print("제작진:")
        if series['credits']['crew']:  # 제작진 정보가 있는 경우에만 출력
            crew = series['credits']['crew'][0]  # 첫 번째 제작진 정보만 출력
            print(f"{crew['name']}")
    
    #10. LOGO
    if 'logos' in series_images:
        for logo in series_images['logos']:
            print("로고 URL:", f"https://image.tmdb.org/t/p/original{logo['file_path']}")

    #11. POSTER_URL
    if 'posters' in series_images:
         for poster in series_images['posters']:
            print("포스터 URL:", f"https://image.tmdb.org/t/p/original{poster['file_path']}")

    #12. TRAILER_URL
    if series_videos is not None:  
        print("예고편 URL:", f"https://www.youtube.com/watch?v={series_videos}")


def main():
    popular_tv_series = fetch_popular_series()

    for series in popular_tv_series:
        series_details = fetch_series_details(series['id'])
        series_images = fetch_series_images(series['id'])
        series_videos = fetch_series_videos(series['id']) 
        print_series_info(series_details, series_images, series_videos) 
        print()

if __name__ == "__main__":
    main()


id: 71026
first_air_date 2014-09-01
제목: 출동 슈퍼윙스
등급: ALL
시즌 수: 15
줄거리: 세상에서 가장 빠른 택배 비행기 호기와 멋진 변신 비행기 친구들 슈퍼윙스가 펼치는 세계 문화탐험 어드벤처 이야기
출연진:
Moon Nam-sook
Chang Eun-sook
Hong So-young
제작진:
Gil Hoon Jung
로고 URL: https://image.tmdb.org/t/p/original/wzicI81VYxDu68CEKqWFlV78TB.png
포스터 URL: https://image.tmdb.org/t/p/original/lu2DNsIQgn1Jb6djHvrqJSUQXXl.jpg

id: 72237
first_air_date 2014-09-03
제목: 최강전사 미니특공대
등급: ALL
시즌 수: 10
줄거리: 수 천년 전, 힘없고 약한 동물들은 스스로를 지키기 위해 특별한 능력을 가진 동물들을 선발하여 특공대를 조직하였다. 그들은 전 세계로 퍼져나가 위험에 빠진 동물들을 구하고 그들의 세력을 점차 확장시켜 나갔다. 오늘날 그들은 뛰어난 과학 기술을 바탕으로 막강한 힘을 키워 세계 평화에 기여하고 있다.
출연진:
Um Sang-hyun
전태열
신용우
제작진:
포스터 URL: https://image.tmdb.org/t/p/original/4SCNu5uw54AgyO7pMWWonuXrUSD.jpg
예고편 URL: https://www.youtube.com/watch?v=t71x-ihYiH0

id: 19845
first_air_date 2003-11-27
제목: 뽀롱뽀롱 뽀로로
등급: ALL
시즌 수: 8
줄거리: 세상이 온통 하얀 눈으로 뒤덮인 어느 추운 나라 작은 섬. 뽀롱뽀롱 숲 속 마을에는 뽀로로, 크롱과 에디, 루피, 포비, 해리, 패티 등의 동물 친구들이 모여 살고 있습니다. 호기심 많은 말썽꾸러기 뽀로로와 크롱 때문에 숲 속 마을에서는 항상 크고 작은 소동이 끓이지 않지만, 친구들은 서로 이해하고 도와가며 함께

KeyboardInterrupt: 